# imports

In [18]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage
)
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import (
    ChatMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# langauge models in langchain

### llms: refers to pure text completion models

In [6]:
llm = Ollama(model="llama3:8b", temperature=0)

In [8]:
# response = llm.invoke("what would be a good company name for a company that makes colorful socks ?")

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product} ?")
chain = prompt | llm
response = chain.invoke({"product": "colorful socks"})

print(type(response))
print(response)

<class 'str'>
What a fun question!

Here are some ideas for a company name that makes colorful socks:

1. **SockSations**: A playful combination of "socks" and "sensations," implying excitement and vibrancy.
2. **HueHub**: A nod to the colorful hues of your socks, with "hub" suggesting a central gathering place or community.
3. **ToeTally Fun**: A whimsical name that references the fun, playful nature of your products.
4. **SoleMates**: A clever play on words, combining "sole" (the bottom of a foot) with "soulmates," implying a connection between people and their favorite socks.
5. **KaleidoKrew**: Inspired by the colorful, swirling patterns of kaleidoscopes, this name suggests a dynamic, creative team behind your company.
6. **SockItToMe**: A cheeky reference to the classic phrase "sock it to me," implying a playful, lighthearted approach to life and fashion.
7. **FiberFrenzy**: This name highlights the colorful fibers used in your socks, with "frenzy" suggesting energy and excitement

### chat model: tuned specifically for having conversations

In [9]:
chat_model = ChatOllama(model="llama3:8b", temperature=0)

In [10]:
messages = [
    HumanMessage(content="what would be a good company name for a company that makes colorful socks ?")
    ]

response = chat_model.invoke(input=messages)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content="What a fun question!\n\nHere are some ideas for a company name that might fit the bill:\n\n1. **SockItToMe**: A playful name that references the idea of putting your best foot forward (or in this case, your most colorful sock).\n2. **HueHub**: A nod to the vibrant colors and hues you'd find in a pair of colorful socks.\n3. **ToeTally Fun**: A whimsical name that captures the lighthearted spirit of wearing fun, colorful socks.\n4. **SoleMates**: A clever play on words that references the idea of finding your perfect match (in this case, a pair of matching colorful socks).\n5. **KaleidoKrew**: A name that references the kaleidoscope of colors and patterns you might find in a pair of colorful socks.\n6. **SockSational**: A playful name that emphasizes the excitement and joy of wearing colorful socks.\n7. **FiberFrenzy**: A name that highlights the creative, artistic aspect of designing and making colorful socks.\n8. **HeelToToe**: A 

# prompt templates

In [11]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product} ?")
prompt.format(product= "colorful socks")

'What is a good name for a company that makes colorful socks ?'

In [19]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])


print(type(chat_prompt))
chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


[SystemMessage(content='you are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

# output parsers

In [24]:
output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

# composing with LCEL: lang chain expression language

In [31]:
chat_model = ChatOllama(model="llama3:8b", temperature=0)
output_parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate.from_template("Generate a list of 5 {text}.\n\n{format_instructions}")
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "you are a helpful assistant."),
#     ("human", "Generate a list of 5 {text}. \n\n {format_instructions}")

# ])

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | chat_model | output_parser

chain.invoke({"text": "colors"})

['Here is the list of 5 colors:\n\nRed', 'Orange', 'Yellow', 'Green', 'Blue']

# Prompts

## Prompt Template
-   used to create a template from a string prompt.

In [14]:
prompt_template = PromptTemplate.from_template("tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")

'tell me a funny joke about chickens.'

## Chat Prompt Template
-   The prompt to chat models/ is a list of chat messages.
-   Each chat message is associated with content, and an additional parameter called role.
-   For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.
-   The ChatPromptTemplate.from_messages static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.

In [17]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI bot. Your name is {name}"),
        ("human", "Hello, how are you doing?."),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}")
    ]
)

messages = chat_prompt_template.format_messages(name="Jarvis", user_input="what is your name?")

print(f"type(messages): {type(messages)}")

messages

type(messages): <class 'list'>


[SystemMessage(content='you are a helpful AI bot. Your name is Jarvis'),
 HumanMessage(content='Hello, how are you doing?.'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='what is your name?')]

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat"),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

## Message Prompts
- used where the chat model supports taking chat message with arbitrary role.

In [51]:
chat_message_prompt_template = ChatMessagePromptTemplate.from_template(role="Jedi", template="May the {subject} be with you")
chat_message_prompt_template.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

## Message Placeholder
- gives full control of what messages to be rendered during formatting.
- used when uncertain of what role should be used and wish to insert a list of messages during formatting.


In [61]:
human_message_template = HumanMessagePromptTemplate.from_template("Summarize our conversation so far in {word_count} words.")

chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"), 
    human_message_template
    ])

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
""" 
)

response = chat_prompt_template.format_prompt(
    conversation=[human_message, ai_message],
    word_count="10"
).to_messages()

print(f"{type(response)}")
response

<class 'list'>


[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='\n1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]